### 새로나온 책(1-10page)
- 제목
- 저자
- 역자
- 출간일
- 페이지
- 정가
- 판가
- 마일리지

In [169]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [170]:
site = 'https://www.hanbit.co.kr'         
url = f'{site}/store/books/new_book_list.html'
res = requests.get(url)

In [171]:
soup = BeautifulSoup(res.text, 'html.parser')

In [172]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [174]:
li = lis[0]
href = li.select_one('.book_tit').find('a')['href']
href

'/store/books/look.php?p_code=B6694310162'

In [175]:
title = li.select_one('.book_tit').find('a').get_text()
title

'하루 한 장 초등과학 365'

In [176]:
author = li.select_one('.book_writer').get_text()
author

'치바 가즈요시 '

In [177]:
sub_url = site + href
sub_res = requests.get(sub_url)
# sub_html = sub_res.text
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [178]:
sub_lis1 = sub_soup.select('ul.info_list > li')
#sub_lis1

In [179]:
translator = sub_lis1[1].get_text().split(':')[1:][0].strip()
translator

'허영은 , 유우종 (감수)'

In [180]:
date = sub_lis1[-4].get_text().split(':')[1].strip()
date

'2021-12-14'

In [181]:
page = sub_lis1[-3].get_text().split(' ')[2]
page

'416'

In [182]:
sub_lis2 = sub_soup.select('div.store_payment_area > fieldset')[0]
#sub_lis2

In [183]:
price = sub_lis2.select('span.pbr > del')[0].get_text()
price

'19,800원'

In [184]:
sel_price = sub_lis2.select('span.pbr > strong')[0].get_text().strip()
sel_price

'17,820'

In [185]:
point = sub_lis2.select('span.pbr')[2].get_text().split(' ')[0]
point

'990점'

In [186]:
def convert_unit(s):
    s = s.replace('원','').replace('점','').replace(',','')
    return f'{int(s):,d}'

In [187]:
lines = []
for li in lis:
    href = li.select_one('.book_tit').find('a')['href']
    title = li.select_one('.book_tit').find('a').get_text()
    author = li.select_one('.book_writer').get_text()

    sub_url = site + href
    sub_res = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis1 = sub_soup.select('ul.info_list > li')
    date = sub_lis1[-4].get_text().split(':')[1].strip()
    page = sub_lis1[-3].get_text().split(' ')[2]
    sub_lis2 = sub_soup.select('div.store_payment_area > fieldset')[0]
    price = sub_lis2.select('span.pbr > del')[0].get_text()
    sel_price = sub_lis2.select('span.pbr > strong')[0].get_text().strip()
    point = sub_lis2.select('span.pbr')[2].get_text().split(' ')[0]
    lines.append([title,author,date,convert_unit(page),convert_unit(price),convert_unit(sel_price),convert_unit(point)])

In [188]:
df = pd.DataFrame(lines, columns=['책제목','저자','출간일','쪽수','정가','판매가','마일리지'])
df.head()

,책제목,저자,출간일,쪽수,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,2021-12-14,416,"19,800","17,820",990
1,"다시, 배우다",폴 김,2021-12-10,304,"16,000","14,400",800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,2021-12-10,376,"22,000","19,800","1,100"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",2021-12-01,384,"18,000","16,200",900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale",2021-11-30,600,"36,000","36,000","1,080"
